# Plot of solutions

In [196]:
import matplotlib as mpb

# show all the matplotlib backends 
mpb.rcsetup.all_backends 

# force matplotlib to use the 'Qt5Agg' backend 
mpb.use(arg='Qt5Agg', force=True)

In [197]:
import numpy as np
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
import cPickle as pickle
import scipy.stats as sp

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [198]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

In [199]:
results_dir = 'data/flightlines_grid_2100pts_100k/both_inversion/num15_d30/inversion_Apr-12-2020.pickle'
directions_dir = 'data/flightlines_grid_2100pts_100k/both_inversion/num15_d30/directions.pickle'

In [200]:
with open(results_dir) as w:
        inversion = pickle.load(w)

In [201]:
with open('data/model.pickle') as w:
        model = pickle.load(w)

In [202]:
with open(directions_dir) as w:
        rd = pickle.load(w)

In [203]:
y = inversion['y']
x = inversion['x']

In [204]:
verts_true = mfun.plot_prisms(model['prisms'], scale=0.001)

In [205]:
k = 1
for a, t, i, d in zip(inversion['amf_results'], inversion['tfa_results'],
                      rd['directions'][0], rd['directions'][1]):
    angle = np.rad2deg(np.arccos(
        np.dot(model['props']['magnetization'],
               a[2][0][0].props['magnetization'])/model['intensity']**2.))
    fig = plt.figure(figsize=(10,9))
    fig.suptitle('Wrong Inc: %.2f$^\circ$    Dec: %.2f$^\circ$  \n    True Inc: %.2f$^\circ$    Dec: %.2f$^\circ$ \n    Diff Inc: %.2f$^\circ$    Dec: %.2f$^\circ$    Angle: %.2f$^\circ$'
                 % (i, d, model['inc'], model['dec'],i - model['inc'],d - model['dec'],angle), fontsize=14)

    # initial approximate
    ax = plt.subplot(2,2,1, projection='3d')

    verts_amf = mfun.plot_prisms(a[2][-1], scale=0.001)
    verts_tfa = mfun.plot_prisms(t[2][-1], scale=0.001)

    # plot sides
    ax.add_collection3d(Poly3DCollection(verts_tfa, alpha=0.3, 
     facecolor='r', linewidths=0.5, edgecolors='k'))
    ax.add_collection3d(Poly3DCollection(verts_true, alpha=0.3, 
     facecolor='b', linewidths=0.5, edgecolors='k'))

    ax.set_xlim(-2.5, 2.5, 100)
    ax.set_ylim(-2.5, 2.5, 100)
    ax.set_zlim(1.8, -0.1, 100)
    ax.tick_params(labelsize= 10)
    ax.set_ylabel('y (km)', fontsize= 14)
    ax.set_xlabel('x (km)', fontsize= 14)
    ax.set_zlabel('z (km)', fontsize= 14)
    ax.view_init(0, 30)
    #ax.text2D(-0.1, 0.1, '(b)', fontsize= 15)

    # residual data and histogram
    ax=plt.subplot(2,2,2)
    #plt.title('Residual', fontsize=20)
    plt.tricontourf(y, x, t[3], 20,
                    cmap='RdBu_r', vmin=np.min(t[3]),
                    vmax=np.min(t[3])*(-1)).ax.tick_params(labelsize=12)
    plt.xlabel('$y$(m)', fontsize=14)
    plt.ylabel('$x$(m)', fontsize=14)
    clb = plt.colorbar(pad=0.01, aspect=20, shrink=1)
    clb.ax.set_title('nT', pad=-275)
    mpl.m2km()
    clb.ax.tick_params(labelsize=13)
    inset = inset_axes(ax, width="40%", height="30%", loc=1, borderpad=0.7)
    mean = np.mean(t[3])
    std = np.std(t[3])
    nbins=30
    n, bins, patches = plt.hist(t[3],bins=nbins, normed=True, facecolor='blue')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    inset.text(0.6, 0.95, "$\mu$ = {:.2f}\n$\sigma$ = {:.2f}".format(mean, std),
                    transform=inset.transAxes, fontsize=6,
                va='top', ha='left', bbox=props)
    gauss = sp.norm.pdf(bins, mean, std)
    plt.plot(bins, gauss, 'k--', linewidth=1., label='Gaussian')
    #ax.text(-6500, 5500, '(a)', fontsize= 15)

    # inverse model view 1
    ax = plt.subplot(2,2,3, projection='3d')

    # plot sides
    ax.add_collection3d(Poly3DCollection(verts_amf, alpha=0.3, 
     facecolor='r', linewidths=0.5, edgecolors='k'))
    ax.add_collection3d(Poly3DCollection(verts_true, alpha=0.3, 
     facecolor='b', linewidths=0.5, edgecolors='k'))

    ax.set_xlim(-2.5, 2.5, 100)
    ax.set_ylim(-2.5, 2.5, 100)
    ax.set_zlim(1.8, -0.1, 100)
    ax.tick_params(labelsize= 10)
    ax.set_ylabel('y (km)', fontsize= 14)
    ax.set_xlabel('x (km)', fontsize= 14)
    ax.set_zlabel('z (km)', fontsize= 14)
    ax.view_init(0, 30)
    #ax.text2D(-0.12, 0.07, '(c)', fontsize= 15)

    # residual data and histogram
    ax=plt.subplot(2,2,4)
    #plt.title('Residual', fontsize=20)
    plt.tricontourf(y, x, a[3], 20,
                    cmap='RdBu_r', vmin=np.min(a[3]),
                    vmax=np.min(a[3])*(-1)).ax.tick_params(labelsize=12)
    plt.xlabel('$y$(m)', fontsize=14)
    plt.ylabel('$x$(m)', fontsize=14)
    clb = plt.colorbar(pad=0.01, aspect=20, shrink=1)
    clb.ax.set_title('nT', pad=-275)
    mpl.m2km()
    clb.ax.tick_params(labelsize=13)
    inset = inset_axes(ax, width="40%", height="30%", loc=1, borderpad=0.7)
    mean = np.mean(a[3])
    std = np.std(a[3])
    nbins=30
    n, bins, patches = plt.hist(a[3],bins=nbins, normed=True, facecolor='blue')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    inset.text(0.6, 0.95, "$\mu$ = {:.2f}\n$\sigma$ = {:.2f}".format(mean, std),
                    transform=inset.transAxes, fontsize=6,
                va='top', ha='left', bbox=props)
    gauss = sp.norm.pdf(bins, mean, std)
    plt.plot(bins, gauss, 'k--', linewidth=1., label='Gaussian')
    #ax.text(-6500, 5500, '(a)', fontsize= 15)
    
    plt.savefig(directions_dir[:-17]+'stereo%d.png' % (k), dpi=300, bbox_inches="tight")
    k += 1
    plt.show()